In [ ]:
#!pip install -qq speechbrain==0.5.16
#!pip install -qq ipython==7.34.0
#!pip install -qq pyannote.audio[separation]==3.3.0

In [ ]:
from pyannote.database.util import load_rttm
from pyannote.audio.core.io import Audio

In [ ]:
audio = Audio()

In [ ]:
uri_new = "truyen"
file_new = "untitled.wav"
mixture_new, sample_rate_new = audio(file=file_new)

In [ ]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained("config.yaml")

# you can safely ignore that various warnings below.

In [ ]:
pipeline.instantiate(
    {
        "segmentation": {
            "min_duration_off": 0.0,
            "threshold": 0.5,
        },
        "clustering": {
            "method": "centroid",
            "threshold": 0.68,
            "min_cluster_size": 60,
        },
        "separation": {
            "leakage_removal": True,
            "asr_collar": 0.0,
        }
    }
)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline.to(device=device)

In [ ]:
from pyannote.metrics.diarization import DiarizationErrorRate
from pyannote.audio.pipelines.utils.hook import ProgressHook

metric = DiarizationErrorRate()

# we use a ProgressHook, which will print progress for each step in the pipeline
with ProgressHook() as hook:
  # apply pipeline on the mixture file and get diarization prediction
  diarization, sources_hat = pipeline({"waveform": mixture, "sample_rate": sample_rate}, hook=hook)